<a href="https://colab.research.google.com/github/CSC-Artificial-Intelligence/Application-of-Neural-Networks/blob/main/nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
# import numpy as np
# import matplotlib.pyplot as plt

In [21]:
# Define the transform with data augmentation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [22]:
# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)

Files already downloaded and verified


Files already downloaded and verified


In [23]:
class_names = [
    "airplane",										
    "automobile",										
    "bird",										
    "cat",										
    "deer",										
    "dog",										
    "frog",										
    "horse",										
    "ship",										
    "truck",
]

In [24]:


# Define a simple CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))
        x = self.pool(nn.ReLU()(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = nn.ReLU()( self.fc1(x))
        x = nn.ReLU()( self.fc2(x))        
        x = self.fc3(x)
        return x


In [25]:
model = CNN()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [27]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(train_loader):
        images, labels = data
        optimizer.zero_grad()  # Clear gradients
        outputs = model(images)  # Forward pass
        loss = loss_function(outputs, labels)
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()


    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

Epoch [1/50], Loss: 1.7747
Epoch [2/50], Loss: 1.5022
Epoch [3/50], Loss: 1.3886
Epoch [4/50], Loss: 1.3015
Epoch [5/50], Loss: 1.2291
Epoch [6/50], Loss: 1.1662
Epoch [7/50], Loss: 1.1080
Epoch [8/50], Loss: 1.0600
Epoch [9/50], Loss: 1.0157
Epoch [10/50], Loss: 0.9710
Epoch [11/50], Loss: 0.9272
Epoch [12/50], Loss: 0.8951
Epoch [13/50], Loss: 0.8594
Epoch [14/50], Loss: 0.8282
Epoch [15/50], Loss: 0.7973
Epoch [16/50], Loss: 0.7692
Epoch [17/50], Loss: 0.7390
Epoch [18/50], Loss: 0.7128
Epoch [19/50], Loss: 0.6872
Epoch [20/50], Loss: 0.6620
Epoch [21/50], Loss: 0.6379
Epoch [22/50], Loss: 0.6112
Epoch [23/50], Loss: 0.5933
Epoch [24/50], Loss: 0.5714
Epoch [25/50], Loss: 0.5454
Epoch [26/50], Loss: 0.5272
Epoch [27/50], Loss: 0.5050
Epoch [28/50], Loss: 0.4853
Epoch [29/50], Loss: 0.4680
Epoch [30/50], Loss: 0.4495
Epoch [31/50], Loss: 0.4295
Epoch [32/50], Loss: 0.4103
Epoch [33/50], Loss: 0.3895
Epoch [34/50], Loss: 0.3759
Epoch [35/50], Loss: 0.3562
Epoch [36/50], Loss: 0.3437
E

In [28]:
torch.save(model.state_dict(), "trained_model.pth")

In [29]:
model = CNN()
model.load_state_dict(torch.load("trained_model.pth"))


/tmp/ipykernel_1893/538299269.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("trained_model.pth"))


<All keys matched successfully>

In [30]:

# Evaluate on test set
model.eval()  # Set model to evaluation mode
correct = 0
total = 0


with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Test Accuracy: 66.89%
